In [1]:
import os
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
from env import get_connection

# custom imports
import wrangle as w

# AQUISITION PHASE OF DATA

In [2]:
### Query used to filter data from database,

# SELECT *

# SELECT bathroomcnt, bedroomcnt, calculatedbathnbr, calculatedfinishedsquarefeet, fips, fullbathcnt, garagecarcnt, 
# garagetotalsqft, heatingorsystemtypeid, latitude, longitude, lotsizesquarefeet, poolcnt, propertycountylandusecode, propertyzoningdesc, 
# rawcensustractandblock, regionidcity, regionidcounty, regionidneighborhood, regionidzip, roomcnt, unitcnt, yearbuilt, numberofstories,  
# taxvaluedollarcnt, assessmentyear, censustractandblock

# FROM properties_2017 AS p17
# LEFT JOIN predictions_2017 AS pr17 ON p17.id = pr17.id
# LEFT JOIN propertylandusetype AS plu ON p17.propertylandusetypeid = plu.propertylandusetypeid
# WHERE plu.propertylandusetypeid = 261 AND YEAR(pr17.transactiondate) = 2017; -- 'Single Family Residential' and transactions in 2017

In [3]:
# changed key id to for sql query above. 
# Parcel_id seems to be a more unique identifier versus the general id, the general id can be ambiguous so it's better to use parcel id to not have mismatched properties with identifier.

# went ahead and changed the query again to return wanted columns instead of generating 65 columns
# adjsted a third time to get columns with only a few null values

In [4]:
# Constants
filename = 'zillow_data.csv'

# Acquire data.
# ----------------------ACQUIRE FUNCTION---------------------------------
def acquire_zillow():

    if os.path.isfile(filename):
        
        return pd.read_csv(filename)
        
    else: 

        query = '''
        SELECT p17.parcelid, bathroomcnt, bedroomcnt, calculatedfinishedsquarefeet, fullbathcnt, latitude, longitude, lotsizesquarefeet,
        regionidcounty, regionidzip, roomcnt, yearbuilt, taxvaluedollarcnt, censustractandblock, fips
        FROM properties_2017 AS p17
        LEFT JOIN predictions_2017 AS pr17 ON p17.parcelid = pr17.parcelid
        LEFT JOIN propertylandusetype AS plu ON p17.propertylandusetypeid = plu.propertylandusetypeid
        WHERE plu.propertylandusetypeid = 261 AND YEAR(pr17.transactiondate) = 2017; -- 'Single Family Residential' and transactions in 2017
        '''

        url = get_connection('zillow')
                
        df = pd.read_sql(query, url)

        # # save to csv
        # df.to_csv(filename,index=False)

        return df 

In [5]:
# acquiring data
df = w.acquire_zillow()
df.head()

,parcelid,bathroomcnt,bedroomcnt,calculatedfinishedsquarefeet,fullbathcnt,latitude,longitude,lotsizesquarefeet,regionidcounty,regionidzip,roomcnt,yearbuilt,taxvaluedollarcnt,censustractandblock,fips
0,14297519,3.5,4.0,3100.0,3.0,33634931.0,-117869207.0,4506.0,1286.0,96978.0,0.0,1998.0,1023282.0,6.059063e+13,6059.0
1,17052889,1.0,2.0,1465.0,1.0,34449266.0,-119281531.0,12647.0,2061.0,97099.0,5.0,1967.0,464000.0,6.111001e+13,6111.0
2,14186244,2.0,3.0,1243.0,2.0,33886168.0,-117823170.0,8432.0,1286.0,97078.0,6.0,1962.0,564778.0,6.059022e+13,6059.0
3,12177905,3.0,4.0,2376.0,3.0,34245180.0,-118240722.0,13038.0,3101.0,96330.0,0.0,1970.0,145143.0,6.037300e+13,6037.0
4,12095076,3.0,4.0,2962.0,3.0,34145202.0,-118179824.0,63000.0,3101.0,96293.0,0.0,1950.0,773303.0,6.037461e+13,6037.0


In [6]:
# checking shape of data
df.shape

(52441, 15)

In [7]:
# checking info of data
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 52441 entries, 0 to 52440
Data columns (total 15 columns):
 #   Column                        Non-Null Count  Dtype  
---  ------                        --------------  -----  
 0   parcelid                      52441 non-null  int64  
 1   bathroomcnt                   52441 non-null  float64
 2   bedroomcnt                    52441 non-null  float64
 3   calculatedfinishedsquarefeet  52359 non-null  float64
 4   fullbathcnt                   52304 non-null  float64
 5   latitude                      52441 non-null  float64
 6   longitude                     52441 non-null  float64
 7   lotsizesquarefeet             52072 non-null  float64
 8   regionidcounty                52441 non-null  float64
 9   regionidzip                   52415 non-null  float64
 10  roomcnt                       52441 non-null  float64
 11  yearbuilt                     52325 non-null  float64
 12  taxvaluedollarcnt             52440 non-null  float64
 13  c

# PREP PHASE OF DATA

In [8]:
# created dup to see how many duplicated rows are in my data frame
dup = df.duplicated() == True
dup.sum()

121

In [9]:
dup = df.duplicated() == False
dup.sum()

52320

In [10]:
dup = df[df.duplicated() == True]
dup.shape

(121, 15)

In [11]:
# double checked for all features again for duplicates
all_duplicates = df[df.duplicated(keep='first')]
all_duplicates

,parcelid,bathroomcnt,bedroomcnt,calculatedfinishedsquarefeet,fullbathcnt,latitude,longitude,lotsizesquarefeet,regionidcounty,regionidzip,roomcnt,yearbuilt,taxvaluedollarcnt,censustractandblock,fips
672,11721753,2.0,3.0,1316.0,2.0,34033180.0,-118355396.0,5672.0,3101.0,95997.0,0.0,1923.0,205123.0,6.037220e+13,6037.0
835,11289917,2.0,3.0,1458.0,2.0,34686163.0,-118113100.0,8284.0,3101.0,97318.0,0.0,1970.0,136104.0,6.037901e+13,6037.0
1196,11705026,1.0,2.0,1421.0,1.0,33999877.0,-118291863.0,6707.0,3101.0,96018.0,0.0,1911.0,35606.0,6.037232e+13,6037.0
1381,14269464,3.0,4.0,2541.0,3.0,33694636.0,-117912245.0,4975.0,1286.0,96958.0,0.0,2003.0,880456.0,6.059064e+13,6059.0
1796,11446756,2.0,3.0,1491.0,2.0,33873860.0,-118338253.0,5000.0,3101.0,96162.0,0.0,1955.0,107110.0,6.037650e+13,6037.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
35442,12621730,3.0,4.0,2354.0,3.0,33807990.0,-118296159.0,6534.0,3101.0,96210.0,0.0,1965.0,97502.0,6.037544e+13,6037.0
36901,10956664,1.0,2.0,1178.0,1.0,34173900.0,-118346284.0,6515.0,3101.0,96437.0,0.0,1943.0,72522.0,6.037311e+13,6037.0
38621,14448410,3.0,3.0,1684.0,3.0,33444723.0,-117638621.0,6660.0,1286.0,96982.0,6.0,1963.0,745361.0,6.059042e+13,6059.0
40595,13066981,2.0,4.0,1533.0,2.0,34028634.0,-117761735.0,6185.0,3101.0,96506.0,0.0,1983.0,366160.0,6.037403e+13,6037.0


In [12]:
# wanted to see if the rows are actually in fact duplicated and I confirmed with a boolean mask to return rows with the same parcelid
# parcelid should be a unique identifier for these properties
dups = df[df['parcelid'] == 11721753]
dups

,parcelid,bathroomcnt,bedroomcnt,calculatedfinishedsquarefeet,fullbathcnt,latitude,longitude,lotsizesquarefeet,regionidcounty,regionidzip,roomcnt,yearbuilt,taxvaluedollarcnt,censustractandblock,fips
671,11721753,2.0,3.0,1316.0,2.0,34033180.0,-118355396.0,5672.0,3101.0,95997.0,0.0,1923.0,205123.0,6.037220e+13,6037.0
672,11721753,2.0,3.0,1316.0,2.0,34033180.0,-118355396.0,5672.0,3101.0,95997.0,0.0,1923.0,205123.0,6.037220e+13,6037.0


In [13]:
# found a parcelid with more than 2 duplicates, realizing multiple with different dates as long features of one property stay in the dataframe should be fine
dups = df[df['parcelid'] == 11991059]
dups

,parcelid,bathroomcnt,bedroomcnt,calculatedfinishedsquarefeet,fullbathcnt,latitude,longitude,lotsizesquarefeet,regionidcounty,regionidzip,roomcnt,yearbuilt,taxvaluedollarcnt,censustractandblock,fips
29557,11991059,6.0,12.0,8469.0,6.0,34072327.0,-118298778.0,NaN,3101.0,95985.0,0.0,1909.0,2485282.0,NaN,6037.0
29558,11991059,6.0,12.0,8469.0,6.0,34072327.0,-118298778.0,NaN,3101.0,95985.0,0.0,1909.0,2485282.0,NaN,6037.0
29559,11991059,6.0,12.0,8469.0,6.0,34072327.0,-118298778.0,NaN,3101.0,95985.0,0.0,1909.0,2485282.0,NaN,6037.0


In [14]:
# checked for all parcelid duplicates and I identified 241 rows
duplicate_parcelids = df[df.duplicated(subset=['parcelid'], keep=False)]
duplicate_parcelids.shape

(241, 15)

In [15]:
52441 - 52320

121

In [16]:
# ~ is used to exclude the rows with duplicate 'parcelid' values from the DataFrame by inverting the boolean Series that identifies duplicates.
df = df[~df.duplicated(keep='first')]
df.shape

(52320, 15)

In [17]:
# double checked for all features again for duplicates
all_duplicates = df[df.duplicated(keep=False)]
all_duplicates

,parcelid,bathroomcnt,bedroomcnt,calculatedfinishedsquarefeet,fullbathcnt,latitude,longitude,lotsizesquarefeet,regionidcounty,regionidzip,roomcnt,yearbuilt,taxvaluedollarcnt,censustractandblock,fips


In [18]:
# double checked by parcelid column again for duplicates
duplicate_parcelids = df[df.duplicated(subset=['parcelid'], keep=False)]
duplicate_parcelids.shape

(0, 15)

In [19]:
df.head()

,parcelid,bathroomcnt,bedroomcnt,calculatedfinishedsquarefeet,fullbathcnt,latitude,longitude,lotsizesquarefeet,regionidcounty,regionidzip,roomcnt,yearbuilt,taxvaluedollarcnt,censustractandblock,fips
0,14297519,3.5,4.0,3100.0,3.0,33634931.0,-117869207.0,4506.0,1286.0,96978.0,0.0,1998.0,1023282.0,6.059063e+13,6059.0
1,17052889,1.0,2.0,1465.0,1.0,34449266.0,-119281531.0,12647.0,2061.0,97099.0,5.0,1967.0,464000.0,6.111001e+13,6111.0
2,14186244,2.0,3.0,1243.0,2.0,33886168.0,-117823170.0,8432.0,1286.0,97078.0,6.0,1962.0,564778.0,6.059022e+13,6059.0
3,12177905,3.0,4.0,2376.0,3.0,34245180.0,-118240722.0,13038.0,3101.0,96330.0,0.0,1970.0,145143.0,6.037300e+13,6037.0
4,12095076,3.0,4.0,2962.0,3.0,34145202.0,-118179824.0,63000.0,3101.0,96293.0,0.0,1950.0,773303.0,6.037461e+13,6037.0


In [20]:
# checked for all null values; i may be losing approximately 37788 columns in df
df.isna().sum()

parcelid                          0
bathroomcnt                       0
bedroomcnt                        0
calculatedfinishedsquarefeet     81
fullbathcnt                     135
latitude                          0
longitude                         0
lotsizesquarefeet               366
regionidcounty                    0
regionidzip                      26
roomcnt                           0
yearbuilt                       114
taxvaluedollarcnt                 1
censustractandblock             121
fips                              0
dtype: int64

I may impute some values for features with less than 500 null values, and drop anything over that.

In [21]:
# df[df['unitcnt'] == 1.0] # may impute 1.0 for unitcnt

In [22]:
# dropping all nulls erases all rows; can't do that, i have to figure out another way to prep data
df2 = df.dropna()
df2

,parcelid,bathroomcnt,bedroomcnt,calculatedfinishedsquarefeet,fullbathcnt,latitude,longitude,lotsizesquarefeet,regionidcounty,regionidzip,roomcnt,yearbuilt,taxvaluedollarcnt,censustractandblock,fips
0,14297519,3.5,4.0,3100.0,3.0,33634931.0,-117869207.0,4506.0,1286.0,96978.0,0.0,1998.0,1023282.0,6.059063e+13,6059.0
1,17052889,1.0,2.0,1465.0,1.0,34449266.0,-119281531.0,12647.0,2061.0,97099.0,5.0,1967.0,464000.0,6.111001e+13,6111.0
2,14186244,2.0,3.0,1243.0,2.0,33886168.0,-117823170.0,8432.0,1286.0,97078.0,6.0,1962.0,564778.0,6.059022e+13,6059.0
3,12177905,3.0,4.0,2376.0,3.0,34245180.0,-118240722.0,13038.0,3101.0,96330.0,0.0,1970.0,145143.0,6.037300e+13,6037.0
4,12095076,3.0,4.0,2962.0,3.0,34145202.0,-118179824.0,63000.0,3101.0,96293.0,0.0,1950.0,773303.0,6.037461e+13,6037.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
52436,12412492,2.0,4.0,1633.0,2.0,33870815.0,-118070858.0,4630.0,3101.0,96204.0,0.0,1962.0,346534.0,6.037555e+13,6037.0
52437,11000655,2.0,2.0,1286.0,2.0,34245368.0,-118282383.0,47405.0,3101.0,96284.0,0.0,1940.0,354621.0,6.037101e+13,6037.0
52438,17239384,2.0,4.0,1612.0,2.0,34300140.0,-118706327.0,12105.0,2061.0,97116.0,7.0,1964.0,67205.0,6.111008e+13,6111.0
52439,12773139,1.0,3.0,1032.0,1.0,34040895.0,-118038169.0,5074.0,3101.0,96480.0,0.0,1954.0,49546.0,6.037434e+13,6037.0


In [23]:
# dropped columns with large null values
# df = df.drop(columns=['garagecarcnt', 'garagetotalsqft', 'heatingorsystemtypeid', 'poolcnt', 'propertyzoningdesc', 'regionidcity', 'regionidneighborhood', 'unitcnt', 'numberofstories']) 

In [24]:
df.head()

,parcelid,bathroomcnt,bedroomcnt,calculatedfinishedsquarefeet,fullbathcnt,latitude,longitude,lotsizesquarefeet,regionidcounty,regionidzip,roomcnt,yearbuilt,taxvaluedollarcnt,censustractandblock,fips
0,14297519,3.5,4.0,3100.0,3.0,33634931.0,-117869207.0,4506.0,1286.0,96978.0,0.0,1998.0,1023282.0,6.059063e+13,6059.0
1,17052889,1.0,2.0,1465.0,1.0,34449266.0,-119281531.0,12647.0,2061.0,97099.0,5.0,1967.0,464000.0,6.111001e+13,6111.0
2,14186244,2.0,3.0,1243.0,2.0,33886168.0,-117823170.0,8432.0,1286.0,97078.0,6.0,1962.0,564778.0,6.059022e+13,6059.0
3,12177905,3.0,4.0,2376.0,3.0,34245180.0,-118240722.0,13038.0,3101.0,96330.0,0.0,1970.0,145143.0,6.037300e+13,6037.0
4,12095076,3.0,4.0,2962.0,3.0,34145202.0,-118179824.0,63000.0,3101.0,96293.0,0.0,1950.0,773303.0,6.037461e+13,6037.0


In [25]:
# checked shape
df.shape

(52320, 15)

In [26]:
# checked to see my resulting df info, there are less large null counts  so now i can probably dropna() and not lose my entire data.
df.isna().sum()

parcelid                          0
bathroomcnt                       0
bedroomcnt                        0
calculatedfinishedsquarefeet     81
fullbathcnt                     135
latitude                          0
longitude                         0
lotsizesquarefeet               366
regionidcounty                    0
regionidzip                      26
roomcnt                           0
yearbuilt                       114
taxvaluedollarcnt                 1
censustractandblock             121
fips                              0
dtype: int64

In [27]:
# dropping all columns of current df
df = df.dropna()
df.shape

(51732, 15)

In [28]:
# checking for null values again
df.isna().sum()

parcelid                        0
bathroomcnt                     0
bedroomcnt                      0
calculatedfinishedsquarefeet    0
fullbathcnt                     0
latitude                        0
longitude                       0
lotsizesquarefeet               0
regionidcounty                  0
regionidzip                     0
roomcnt                         0
yearbuilt                       0
taxvaluedollarcnt               0
censustractandblock             0
fips                            0
dtype: int64

In [29]:
# checking shape, looks like my dataframe was reduced to approximately 600 rows that contained null values
df.shape

(51732, 15)

In [30]:
# make sure the math is mathing
52320 - 51732

588

In [31]:
# looking over the column names see if I can change these to be more pythonic
df.dtypes

parcelid                          int64
bathroomcnt                     float64
bedroomcnt                      float64
calculatedfinishedsquarefeet    float64
fullbathcnt                     float64
latitude                        float64
longitude                       float64
lotsizesquarefeet               float64
regionidcounty                  float64
regionidzip                     float64
roomcnt                         float64
yearbuilt                       float64
taxvaluedollarcnt               float64
censustractandblock             float64
fips                            float64
dtype: object

In [32]:
# renamed appropriately
df = df.rename(columns={'bedroomcnt': 'bedrooms', 'bathroomcnt': 'bathrooms', 
                            'calculatedfinishedsquarefeet': 'area', 'taxvaluedollarcnt': 'home_value', 
                            'yearbuilt': 'year_built', 'fullbathcnt' : 'full_bath_cnt', 'lotsizesquarefeet' : 'lot_area', 'regionidcounty' : 'region_id_county',
                            'roomcnt' : 'room_cnt', 'censustractandblock' : 'census_tract_and_block'})

In [33]:
# looking at dataframe to ensure changes were made to columns
df.head()

,parcelid,bathrooms,bedrooms,area,full_bath_cnt,latitude,longitude,lot_area,region_id_county,regionidzip,room_cnt,year_built,home_value,census_tract_and_block,fips
0,14297519,3.5,4.0,3100.0,3.0,33634931.0,-117869207.0,4506.0,1286.0,96978.0,0.0,1998.0,1023282.0,6.059063e+13,6059.0
1,17052889,1.0,2.0,1465.0,1.0,34449266.0,-119281531.0,12647.0,2061.0,97099.0,5.0,1967.0,464000.0,6.111001e+13,6111.0
2,14186244,2.0,3.0,1243.0,2.0,33886168.0,-117823170.0,8432.0,1286.0,97078.0,6.0,1962.0,564778.0,6.059022e+13,6059.0
3,12177905,3.0,4.0,2376.0,3.0,34245180.0,-118240722.0,13038.0,3101.0,96330.0,0.0,1970.0,145143.0,6.037300e+13,6037.0
4,12095076,3.0,4.0,2962.0,3.0,34145202.0,-118179824.0,63000.0,3101.0,96293.0,0.0,1950.0,773303.0,6.037461e+13,6037.0


In [34]:
# can i change the dtypes from flot to int for most of these?
df.dtypes

parcelid                    int64
bathrooms                 float64
bedrooms                  float64
area                      float64
full_bath_cnt             float64
latitude                  float64
longitude                 float64
lot_area                  float64
region_id_county          float64
regionidzip               float64
room_cnt                  float64
year_built                float64
home_value                float64
census_tract_and_block    float64
fips                      float64
dtype: object

In [35]:
# used list of columns to change datatyoes from float to int.
int_columns = ['fips', 'year_built', 'home_value', 'area', 'bedrooms', 'full_bath_cnt', 'room_cnt', 'region_id_county', 'regionidzip', 'lot_area']
df[int_columns] = df[int_columns].astype(int)

In [36]:
# dtypes are now appropriately set
df.dtypes

parcelid                    int64
bathrooms                 float64
bedrooms                    int64
area                        int64
full_bath_cnt               int64
latitude                  float64
longitude                 float64
lot_area                    int64
region_id_county            int64
regionidzip                 int64
room_cnt                    int64
year_built                  int64
home_value                  int64
census_tract_and_block    float64
fips                        int64
dtype: object

In [37]:
# looking for additional prep steps
df.head()

,parcelid,bathrooms,bedrooms,area,full_bath_cnt,latitude,longitude,lot_area,region_id_county,regionidzip,room_cnt,year_built,home_value,census_tract_and_block,fips
0,14297519,3.5,4,3100,3,33634931.0,-117869207.0,4506,1286,96978,0,1998,1023282,6.059063e+13,6059
1,17052889,1.0,2,1465,1,34449266.0,-119281531.0,12647,2061,97099,5,1967,464000,6.111001e+13,6111
2,14186244,2.0,3,1243,2,33886168.0,-117823170.0,8432,1286,97078,6,1962,564778,6.059022e+13,6059
3,12177905,3.0,4,2376,3,34245180.0,-118240722.0,13038,3101,96330,0,1970,145143,6.037300e+13,6037
4,12095076,3.0,4,2962,3,34145202.0,-118179824.0,63000,3101,96293,0,1950,773303,6.037461e+13,6037


In [38]:
# Mapped state names to fips code

fips_to_state = {
    6037: 'California',
    6059: 'California',
    6111: 'California',
    # Add more mappings for other states as needed
    }

# Mapped county names to fips code
fips_to_county = {
    6037: 'Los Angeles County',
    6059: 'Orange County',
    6111: 'Ventura County',
    # Add more mappings for other counties as needed
    }

# Use the map method to create new 'county' and 'state' columns based on 'fips' column
df['state'] = df['fips'].map(fips_to_state)
df['county'] = df['fips'].map(fips_to_county)

In [39]:
df.head()

,parcelid,bathrooms,bedrooms,area,full_bath_cnt,latitude,longitude,lot_area,region_id_county,regionidzip,room_cnt,year_built,home_value,census_tract_and_block,fips,state,county
0,14297519,3.5,4,3100,3,33634931.0,-117869207.0,4506,1286,96978,0,1998,1023282,6.059063e+13,6059,California,Orange County
1,17052889,1.0,2,1465,1,34449266.0,-119281531.0,12647,2061,97099,5,1967,464000,6.111001e+13,6111,California,Ventura County
2,14186244,2.0,3,1243,2,33886168.0,-117823170.0,8432,1286,97078,6,1962,564778,6.059022e+13,6059,California,Orange County
3,12177905,3.0,4,2376,3,34245180.0,-118240722.0,13038,3101,96330,0,1970,145143,6.037300e+13,6037,California,Los Angeles County
4,12095076,3.0,4,2962,3,34145202.0,-118179824.0,63000,3101,96293,0,1950,773303,6.037461e+13,6037,California,Los Angeles County


In [40]:
def clean_zillow(df):

    """
    Cleans the Zillow data.
    
    Args:
    - df (DataFrame): Raw Zillow data.
    
    Returns:
    - cleaned_df (DataFrame): Cleaned Zillow data.
    """

    # Drop rows with duplicates
    df = df[~df.duplicated(keep='first')]
    
    # Drop rows with missing/null values in any column
    df = df.dropna()

    # Rename columns
    df = df.rename(columns={'bedroomcnt': 'bedrooms', 'bathroomcnt':'bathrooms', 
                            'calculatedfinishedsquarefeet': 'area', 'taxvaluedollarcnt': 'home_value',
                            'yearbuilt': 'year_built', 'fullbathcnt' : 'full_bath_cnt',
                            'lotsizesquarefeet' : 'lot_area', 'regionidcounty' : 'region_id_county',
                            'roomcnt' : 'room_cnt', 'censustractandblock' : 'census_tract_and_block'})

    # Convert selected columns to integer type
    int_columns = ['fips', 'year_built', 'home_value', 'area', 'bedrooms', 'full_bath_cnt', 'room_cnt',
                   'region_id_county', 'regionidzip', 'lot_area']
    df[int_columns] = df[int_columns].astype(int)

    fips_to_state = {
        6037: 'California',
        6059: 'California',
        6111: 'California',
        # Add more mappings for other states as needed
        }

    # Mapped county names to fips code
    fips_to_county = {
        6037: 'Los Angeles County',
        6059: 'Orange County',
        6111: 'Ventura County',
        # Add more mappings for other counties as needed
        }
    
    # Use the map method to create new 'county' and 'state' columns based on 'fips' column
    df['state'] = df['fips'].map(fips_to_state)
    df['county'] = df['fips'].map(fips_to_county)
    
    return df

In [41]:
df = w.clean_zillow(df)
df.head()

,parcelid,bathrooms,bedrooms,area,full_bath_cnt,latitude,longitude,lot_area,region_id_county,regionidzip,room_cnt,year_built,home_value,census_tract_and_block,fips,state,county
0,14297519,3.5,4,3100,3,33634931.0,-117869207.0,4506,1286,96978,0,1998,1023282,6.059063e+13,6059,California,Orange County
1,17052889,1.0,2,1465,1,34449266.0,-119281531.0,12647,2061,97099,5,1967,464000,6.111001e+13,6111,California,Ventura County
2,14186244,2.0,3,1243,2,33886168.0,-117823170.0,8432,1286,97078,6,1962,564778,6.059022e+13,6059,California,Orange County
3,12177905,3.0,4,2376,3,34245180.0,-118240722.0,13038,3101,96330,0,1970,145143,6.037300e+13,6037,California,Los Angeles County
4,12095076,3.0,4,2962,3,34145202.0,-118179824.0,63000,3101,96293,0,1950,773303,6.037461e+13,6037,California,Los Angeles County


In [44]:
df.shape # checked

(51732, 17)

In [43]:
# dtypes includes objects, but they can be dropped for analysis of numerical features for this regression project.
df.dtypes

parcelid                    int64
bathrooms                 float64
bedrooms                    int64
area                        int64
full_bath_cnt               int64
latitude                  float64
longitude                 float64
lot_area                    int64
region_id_county            int64
regionidzip                 int64
room_cnt                    int64
year_built                  int64
home_value                  int64
census_tract_and_block    float64
fips                        int64
state                      object
county                     object
dtype: object

# EXPLORE PHASE OF DATA